# FAIDM Group Project: Complete CRISP-DM Analysis
## Student Performance Prediction & Clustering
### Open University Learning Analytics Dataset (OULAD)

---

**Module:** WM9QG-15 Fundamentals of AI and Data Mining

**Dataset:** OULAD Mega Table  
- **Each row = one student enrolled in one module presentation**
- Contains: demographics, VLE engagement, assessment scores, registration info

**Tasks:**
1. **Classification**: Predict student success (Pass/Distinction) vs failure (Fail/Withdrawn)
2. **Clustering**: Segment students by engagement patterns

---

# 0️⃣ CRISP-DM Overview

This notebook follows the **CRISP-DM** (Cross-Industry Standard Process for Data Mining) methodology:

| Phase | Description | Sections |
|-------|-------------|----------|
| 1. Business Understanding | Define objectives and goals | 2️⃣ |
| 2. Data Understanding | Explore, visualize, identify quality issues | 3️⃣ |
| 3. Data Preparation | Clean, transform, engineer features | 4️⃣ |
| 4. Modelling | Build classification and clustering models | 5️⃣ 6️⃣ |
| 5. Evaluation | Assess model performance | 7️⃣ |
| 6. Deployment | Recommendations for implementation | 8️⃣ |

**Prerequisite:** Run `Create_Mega_Table.ipynb` first to generate `oulad_mega_table.csv`

---

# 1️⃣ Setup and Libraries

In [ ]:
# =============================================================================
# IMPORT LIBRARIES
# =============================================================================

# Core data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Modelling
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Evaluation metrics
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, classification_report, confusion_matrix,
                             roc_curve, silhouette_score, davies_bouldin_score)

# Scipy for statistics
from scipy import stats

# Settings
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-whitegrid')

print("✓ All libraries loaded successfully!")
print("-" * 60)

---

# 2️⃣ Phase 1: Business Understanding

## 2.1 Business Context

The **Open University (OU)** is the largest university in the UK for undergraduate education, with a focus on distance learning. The university faces challenges with:
- **High dropout rates** in online courses
- **Late identification** of struggling students
- **Limited resources** for personalized intervention

## 2.2 Business Objectives

1. **Identify at-risk students early** (within first 2-4 weeks) for timely intervention
2. **Understand engagement patterns** that differentiate successful vs unsuccessful students
3. **Segment students** into groups for targeted support strategies

## 2.3 Data Mining Goals

| Task | Type | Goal | Success Metric |
|------|------|------|----------------|
| Task 1 | Classification | Predict Pass/Distinction vs Fail/Withdrawn | AUC-ROC > 0.75, Accuracy > 70% |
| Task 2 | Clustering | Segment students into meaningful groups | Silhouette Score > 0.2 |

## 2.4 Success Criteria

- Model can identify at-risk students with **>75% AUC-ROC**
- Early engagement features (first 2 weeks) have **predictive power**
- Clusters are **interpretable** and map to distinct outcomes
- Recommendations are **actionable** for university staff

---

# 3️⃣ Phase 2: Data Understanding

This phase involves thorough exploration of the data to understand its structure, quality, and characteristics.

## 3.1 Load Data

In [ ]:
# =============================================================================
# LOAD THE MEGA TABLE
# =============================================================================
# The mega table was created by merging all 7 OULAD tables
# Each row represents ONE STUDENT enrolled in ONE MODULE PRESENTATION
# =============================================================================

# UPDATE THIS PATH if your file is in a different location
DATA_PATH = 'oulad_mega_table.csv'

print(f"Loading data from: {DATA_PATH}")
print("-" * 60)

df = pd.read_csv(DATA_PATH)

print(f"✓ Data loaded successfully!")
print(f"\n📊 Dataset Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"\n📌 Each row = one student's enrollment in one module")
print("-" * 60)

## 3.2 Initial Data Inspection

In [ ]:
# =============================================================================
# FIRST LOOK AT THE DATA
# =============================================================================
# .head() shows the first 5 rows to understand data structure
# =============================================================================

print("First 5 rows of the dataset:")
print("-" * 60)
df.head()

In [ ]:
# =============================================================================
# LIST ALL COLUMNS
# =============================================================================
# Understanding what variables we have available
# =============================================================================

print("All columns in the mega table:")
print("-" * 60)

for i, col in enumerate(df.columns, 1):
    dtype = df[col].dtype
    print(f"{i:2}. {col} ({dtype})")

print("-" * 60)
print(f"Total: {len(df.columns)} columns")

In [ ]:
# =============================================================================
# DATA TYPES SUMMARY
# =============================================================================
# Checking which columns are numerical vs categorical
# =============================================================================

print("Data types summary:")
print("-" * 60)
print(df.dtypes.value_counts())
print("-" * 60)

# Separate numerical and categorical
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

print(f"\nNumerical columns: {len(numerical_cols)}")
print(f"Categorical columns: {len(categorical_cols)}")

## 3.3 Missing Values Analysis

In [ ]:
# =============================================================================
# MISSING VALUES ANALYSIS
# =============================================================================
# Identifying which columns have missing data and how much
# This is crucial for deciding imputation strategies
# =============================================================================

print("Missing Values Analysis:")
print("-" * 60)

missing = df.isna().sum()
missing_pct = (missing / len(df) * 100).round(2)

missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct,
    'Data Type': df.dtypes
})

# Show only columns with missing values, sorted by count
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

if len(missing_df) > 0:
    print(missing_df)
    print("-" * 60)
    print(f"\n⚠️ {len(missing_df)} columns have missing values")
    print("\n📝 Interpretation:")
    print("   - Missing VLE data = students who never accessed the VLE (fill with 0)")
    print("   - Missing assessment data = students who didn't submit (fill with 0)")
    print("   - Missing imd_band = unknown socioeconomic status (create indicator)")
else:
    print("✓ No missing values found!")
print("-" * 60)

In [ ]:
# =============================================================================
# VISUALIZE MISSING VALUES
# =============================================================================

if len(missing_df) > 0:
    plt.figure(figsize=(12, 6))
    missing_df['Missing %'].head(15).plot(kind='barh', color='coral')
    plt.xlabel('Missing Percentage (%)')
    plt.title('Top 15 Columns with Missing Values', fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

## 3.4 Duplicate Rows Check

In [ ]:
# =============================================================================
# CHECK FOR DUPLICATE ROWS
# =============================================================================
# Duplicates can skew analysis and model training
# =============================================================================

print("Duplicate Rows Check:")
print("-" * 60)

n_duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {n_duplicates}")

if n_duplicates > 0:
    print(f"⚠️ {n_duplicates} duplicates found ({n_duplicates/len(df)*100:.2f}%)")
    print("   These will be removed during data preparation.")
else:
    print("✓ No duplicate rows found")
print("-" * 60)

## 3.5 Target Variable Analysis

In [ ]:
# =============================================================================
# TARGET VARIABLE: final_result
# =============================================================================
# This is what we want to predict
# Four categories: Pass, Distinction, Fail, Withdrawn
# =============================================================================

print("Target Variable Analysis (final_result):")
print("-" * 60)

target_counts = df['final_result'].value_counts()
target_pct = df['final_result'].value_counts(normalize=True) * 100

print("Distribution:")
for result in target_counts.index:
    print(f"  {result}: {target_counts[result]:,} ({target_pct[result]:.1f}%)")

print("-" * 60)
print(f"\n📊 Success (Pass + Distinction): {target_pct.get('Pass', 0) + target_pct.get('Distinction', 0):.1f}%")
print(f"📊 Failure (Fail + Withdrawn): {target_pct.get('Fail', 0) + target_pct.get('Withdrawn', 0):.1f}%")
print("\n⚠️ Class imbalance detected - will use stratified sampling")

In [ ]:
# =============================================================================
# VISUALIZE TARGET DISTRIBUTION
# =============================================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Color scheme: green for success, red/gray for failure
colors = {'Pass': '#2ecc71', 'Distinction': '#3498db', 'Fail': '#e74c3c', 'Withdrawn': '#95a5a6'}
order = ['Pass', 'Distinction', 'Fail', 'Withdrawn']
color_list = [colors[o] for o in order]

# Bar chart
target_counts.reindex(order).plot(kind='bar', ax=axes[0], color=color_list, edgecolor='black')
axes[0].set_title('Final Results Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Final Result')
axes[0].set_ylabel('Number of Students')
axes[0].tick_params(axis='x', rotation=0)

# Add value labels on bars
for i, v in enumerate(target_counts.reindex(order)):
    axes[0].text(i, v + 200, f'{v:,}', ha='center', fontsize=10)

# Pie chart
axes[1].pie(target_counts.reindex(order), labels=order, autopct='%1.1f%%', 
            colors=color_list, startangle=90, explode=[0.02]*4)
axes[1].set_title('Final Results Proportion', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 3.6 Numerical Variables: Summary Statistics

In [ ]:
# =============================================================================
# SUMMARY STATISTICS FOR NUMERICAL VARIABLES
# =============================================================================
# .describe() provides count, mean, std, min, 25%, 50%, 75%, max
# =============================================================================

print("Summary Statistics (Numerical Variables):")
print("-" * 60)
df.describe().T.round(2)

## 3.7 Distribution Analysis (Histograms)

In [ ]:
# =============================================================================
# DISTRIBUTION OF KEY NUMERICAL VARIABLES
# =============================================================================
# Histograms help us understand:
# - Shape of distribution (normal, skewed, bimodal)
# - Presence of outliers
# - Need for transformation
# =============================================================================

# Select key variables to visualize
key_vars = ['vle_total_clicks', 'vle_active_days', 'assess_score_mean', 
            'vle_early_clicks', 'studied_credits', 'num_of_prev_attempts']
key_vars = [v for v in key_vars if v in df.columns]

print("Distribution of Key Variables:")
print("-" * 60)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, var in enumerate(key_vars):
    ax = axes[i]
    
    # Histogram with KDE
    df[var].hist(bins=50, ax=ax, color='steelblue', edgecolor='black', alpha=0.7, density=True)
    df[var].plot(kind='kde', ax=ax, color='red', linewidth=2)
    
    # Calculate skewness
    skewness = df[var].skew()
    
    ax.set_title(f'{var}\n(Skewness: {skewness:.2f})', fontweight='bold')
    ax.set_xlabel(var)
    ax.set_ylabel('Density')
    
    # Add interpretation
    if abs(skewness) > 1:
        ax.annotate('Highly skewed', xy=(0.7, 0.9), xycoords='axes fraction', color='red')

plt.tight_layout()
plt.show()

print("\n📝 Interpretation:")
print("   - Skewness > 1: Right-skewed (long tail to the right)")
print("   - Skewness < -1: Left-skewed (long tail to the left)")
print("   - |Skewness| < 0.5: Approximately symmetric")

## 3.8 Skewness Analysis

In [ ]:
# =============================================================================
# SKEWNESS ANALYSIS FOR ALL NUMERICAL COLUMNS
# =============================================================================
# Skewness measures asymmetry of the distribution
# Highly skewed variables may need transformation
# =============================================================================

print("Skewness Analysis:")
print("-" * 60)

skewness = df[numerical_cols].skew().sort_values(ascending=False)

print("Most positively skewed (right-tailed):")
for var, skew in skewness.head(5).items():
    print(f"  {var}: {skew:.2f}")

print("\nMost negatively skewed (left-tailed):")
for var, skew in skewness.tail(5).items():
    print(f"  {var}: {skew:.2f}")

# Count highly skewed
highly_skewed = (skewness.abs() > 1).sum()
print(f"\n⚠️ {highly_skewed} variables are highly skewed (|skewness| > 1)")
print("-" * 60)

## 3.9 Outlier Detection

In [ ]:
# =============================================================================
# OUTLIER DETECTION USING IQR METHOD
# =============================================================================
# IQR (Interquartile Range) method:
# - Q1 = 25th percentile, Q3 = 75th percentile
# - IQR = Q3 - Q1
# - Outliers: values < Q1 - 1.5*IQR or > Q3 + 1.5*IQR
# =============================================================================

print("Outlier Detection (IQR Method):")
print("-" * 60)

def count_outliers_iqr(series):
    """Count outliers using IQR method"""
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = ((series < lower_bound) | (series > upper_bound)).sum()
    return outliers, lower_bound, upper_bound

outlier_results = []
for col in key_vars:
    n_outliers, lb, ub = count_outliers_iqr(df[col].dropna())
    pct = n_outliers / len(df) * 100
    outlier_results.append({
        'Variable': col,
        'Outliers': n_outliers,
        'Percentage': f'{pct:.1f}%',
        'Lower Bound': f'{lb:.1f}',
        'Upper Bound': f'{ub:.1f}'
    })

outlier_df = pd.DataFrame(outlier_results)
print(outlier_df.to_string(index=False))
print("-" * 60)

In [ ]:
# =============================================================================
# VISUALIZE OUTLIERS WITH BOX PLOTS
# =============================================================================

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, var in enumerate(key_vars):
    ax = axes[i]
    df.boxplot(column=var, ax=ax)
    ax.set_title(f'{var}', fontweight='bold')
    ax.set_ylabel('Value')

plt.suptitle('Box Plots Showing Outliers', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("📝 Box Plot Interpretation:")
print("   - Box: 25th to 75th percentile (IQR)")
print("   - Line in box: Median (50th percentile)")
print("   - Whiskers: 1.5 × IQR from box")
print("   - Points beyond whiskers: Outliers")

## 3.10 Categorical Variables Analysis

In [ ]:
# =============================================================================
# CATEGORICAL VARIABLES DISTRIBUTION
# =============================================================================

cat_vars = ['gender', 'age_band', 'highest_education', 'imd_band', 'disability', 'region']
cat_vars = [v for v in cat_vars if v in df.columns]

print("Categorical Variables Distribution:")
print("-" * 60)

for var in cat_vars:
    print(f"\n{var}:")
    print(df[var].value_counts())
print("-" * 60)

In [ ]:
# =============================================================================
# VISUALIZE CATEGORICAL VARIABLES
# =============================================================================

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, var in enumerate(cat_vars[:6]):
    ax = axes[i]
    df[var].value_counts().plot(kind='bar', ax=ax, color='steelblue', edgecolor='black')
    ax.set_title(f'{var} Distribution', fontweight='bold')
    ax.set_ylabel('Count')
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 3.11 Feature Relationships with Target

In [ ]:
# =============================================================================
# NUMERICAL FEATURES VS TARGET
# =============================================================================
# Box plots showing how feature distributions differ by outcome
# =============================================================================

print("Feature Distributions by Outcome:")
print("-" * 60)

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, var in enumerate(key_vars):
    ax = axes[i]
    order = ['Pass', 'Distinction', 'Fail', 'Withdrawn']
    df.boxplot(column=var, by='final_result', ax=ax, positions=[1, 2, 3, 4])
    ax.set_title(f'{var} by Outcome', fontweight='bold')
    ax.set_xlabel('Final Result')
    ax.set_ylabel(var)
    plt.suptitle('')

plt.tight_layout()
plt.show()

print("📝 Observation: Students who Pass/Distinction tend to have higher VLE engagement and assessment scores")

In [ ]:
# =============================================================================
# CATEGORICAL FEATURES VS TARGET (Stacked Bar Charts)
# =============================================================================

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, var in enumerate(cat_vars[:6]):
    ax = axes[i]
    ct = pd.crosstab(df[var], df['final_result'], normalize='index') * 100
    order = [c for c in ['Pass', 'Distinction', 'Fail', 'Withdrawn'] if c in ct.columns]
    ct[order].plot(kind='bar', stacked=True, ax=ax,
                   color=['#2ecc71', '#3498db', '#e74c3c', '#95a5a6'])
    ax.set_title(f'Outcome by {var}', fontweight='bold')
    ax.set_ylabel('Percentage')
    ax.tick_params(axis='x', rotation=45)
    ax.legend(loc='upper right', fontsize=8)

plt.tight_layout()
plt.show()

## 3.12 Scatter Plots (Feature Relationships)

In [ ]:
# =============================================================================
# SCATTER PLOTS BETWEEN KEY FEATURES
# =============================================================================
# Scatter plots help identify:
# - Linear relationships between variables
# - Clusters or patterns
# - Outliers
# =============================================================================

print("Scatter Plots Between Key Features:")
print("-" * 60)

# Create binary target for coloring
df['target_binary'] = df['final_result'].apply(lambda x: 1 if x in ['Pass', 'Distinction'] else 0)

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Plot 1: Total clicks vs Active days
ax = axes[0, 0]
scatter = ax.scatter(df['vle_active_days'], df['vle_total_clicks'], 
                     c=df['target_binary'], cmap='RdYlGn', alpha=0.3, s=10)
ax.set_xlabel('VLE Active Days')
ax.set_ylabel('VLE Total Clicks')
ax.set_title('Total Clicks vs Active Days', fontweight='bold')
plt.colorbar(scatter, ax=ax, label='Success (1=Pass/Dist)')

# Plot 2: Assessment score vs Total clicks
ax = axes[0, 1]
scatter = ax.scatter(df['vle_total_clicks'], df['assess_score_mean'], 
                     c=df['target_binary'], cmap='RdYlGn', alpha=0.3, s=10)
ax.set_xlabel('VLE Total Clicks')
ax.set_ylabel('Assessment Score Mean')
ax.set_title('Assessment Score vs Total Clicks', fontweight='bold')
plt.colorbar(scatter, ax=ax, label='Success (1=Pass/Dist)')

# Plot 3: Early clicks vs Total clicks
if 'vle_early_clicks' in df.columns:
    ax = axes[1, 0]
    scatter = ax.scatter(df['vle_early_clicks'], df['vle_total_clicks'], 
                         c=df['target_binary'], cmap='RdYlGn', alpha=0.3, s=10)
    ax.set_xlabel('VLE Early Clicks (First 2 Weeks)')
    ax.set_ylabel('VLE Total Clicks')
    ax.set_title('Early Engagement vs Total Engagement', fontweight='bold')
    plt.colorbar(scatter, ax=ax, label='Success (1=Pass/Dist)')

# Plot 4: Assessment score vs Active days
ax = axes[1, 1]
scatter = ax.scatter(df['vle_active_days'], df['assess_score_mean'], 
                     c=df['target_binary'], cmap='RdYlGn', alpha=0.3, s=10)
ax.set_xlabel('VLE Active Days')
ax.set_ylabel('Assessment Score Mean')
ax.set_title('Assessment Score vs Active Days', fontweight='bold')
plt.colorbar(scatter, ax=ax, label='Success (1=Pass/Dist)')

plt.tight_layout()
plt.show()

print("📝 Observations:")
print("   - Green points (success) cluster in high-engagement regions")
print("   - Red points (failure) cluster in low-engagement regions")
print("   - Clear separation suggests these features have predictive power")

## 3.13 Correlation Analysis

In [ ]:
# =============================================================================
# CORRELATION MATRIX
# =============================================================================
# Correlation measures linear relationship between variables
# Range: -1 (perfect negative) to +1 (perfect positive)
# =============================================================================

print("Correlation Analysis:")
print("-" * 60)

# Select numerical columns for correlation
corr_cols = key_vars + ['target_binary']
corr_cols = [c for c in corr_cols if c in df.columns]

corr_matrix = df[corr_cols].corr()

# Visualize
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f',
            square=True, linewidths=0.5)
plt.title('Correlation Matrix (Key Variables)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n📝 Correlation Interpretation:")
print("   - |r| > 0.7: Strong correlation")
print("   - |r| 0.4-0.7: Moderate correlation")
print("   - |r| < 0.4: Weak correlation")

In [ ]:
# =============================================================================
# CORRELATION WITH TARGET
# =============================================================================

print("Correlation with Target (Success):")
print("-" * 60)

# Get all numerical columns
all_numerical = df.select_dtypes(include=[np.number]).columns.tolist()
all_numerical = [c for c in all_numerical if c != 'target_binary' and c != 'id_student']

target_corr = df[all_numerical + ['target_binary']].corr()['target_binary'].drop('target_binary')
target_corr_sorted = target_corr.abs().sort_values(ascending=False)

print("Top 15 features correlated with success:")
for i, (feat, corr) in enumerate(target_corr_sorted.head(15).items(), 1):
    direction = '+' if target_corr[feat] > 0 else '-'
    print(f"  {i:2}. {feat}: {direction}{corr:.4f}")
print("-" * 60)

## 3.14 Analysis by Module

In [ ]:
# =============================================================================
# ANALYSIS BY MODULE
# =============================================================================
# Do patterns differ across modules?
# =============================================================================

if 'code_module' in df.columns:
    print("Success Rate by Module:")
    print("-" * 60)
    
    module_stats = df.groupby('code_module').agg({
        'target_binary': ['mean', 'count'],
        'vle_total_clicks': 'mean',
        'assess_score_mean': 'mean'
    }).round(2)
    
    module_stats.columns = ['Success Rate', 'N Students', 'Avg Clicks', 'Avg Score']
    module_stats['Success Rate'] = (module_stats['Success Rate'] * 100).round(1).astype(str) + '%'
    print(module_stats)
    print("-" * 60)

## 3.15 Data Understanding Summary

In [ ]:
# =============================================================================
# DATA UNDERSTANDING SUMMARY
# =============================================================================

print("=" * 70)
print("                    DATA UNDERSTANDING SUMMARY")
print("=" * 70)

print(f"""
📊 DATASET OVERVIEW
────────────────────
• Rows: {df.shape[0]:,} (student-module enrollments)
• Columns: {df.shape[1]}
• Numerical features: {len(numerical_cols)}
• Categorical features: {len(categorical_cols)}

🎯 TARGET VARIABLE (final_result)
──────────────────────────────────
• Pass: {target_pct.get('Pass', 0):.1f}%
• Distinction: {target_pct.get('Distinction', 0):.1f}%
• Fail: {target_pct.get('Fail', 0):.1f}%
• Withdrawn: {target_pct.get('Withdrawn', 0):.1f}%
• Class imbalance: YES (need stratified sampling)

⚠️ DATA QUALITY ISSUES
───────────────────────
• Missing values: {len(missing_df)} columns affected
• Duplicate rows: {n_duplicates}
• Highly skewed variables: {highly_skewed}
• Outliers: Present in engagement metrics

📈 KEY FINDINGS
─────────────────
• VLE engagement strongly correlates with success
• Assessment scores strongly correlate with success
• Early engagement (first 2 weeks) shows predictive potential
• Clear separation between success/failure in scatter plots

📝 IMPLICATIONS FOR DATA PREPARATION
──────────────────────────────────────
• Fill missing VLE/assessment with 0 (no activity)
• Create missing indicator for imd_band
• Consider outlier handling (capping)
• Engineer new features (ratios, bins)
• Encode categorical variables
""")
print("=" * 70)

---

# 4️⃣ Phase 3: Data Preparation

This phase transforms raw data into a clean, analysis-ready format.

## 4.1 Create Working Copy

In [ ]:
# =============================================================================
# CREATE WORKING COPY
# =============================================================================
# Always work on a copy to preserve original data
# =============================================================================

print("Step 1: Create working copy")
print("-" * 60)

df_prep = df.copy()
print(f"✓ Created copy: {df_prep.shape[0]:,} rows × {df_prep.shape[1]} columns")
print("-" * 60)

## 4.2 Handle Missing Values

In [ ]:
# =============================================================================
# HANDLE MISSING VALUES
# =============================================================================
# Strategy:
# 1. imd_band: Create missing indicator, then fill with 'Unknown'
# 2. VLE columns: Fill with 0 (no activity)
# 3. Assessment columns: Fill with 0 (no submissions)
# 4. Remaining numerical: Impute with median
# =============================================================================

print("Step 2: Handle missing values")
print("-" * 60)

# 2a. Create missing indicator for imd_band (as taught in class)
if 'imd_band' in df_prep.columns:
    df_prep['imd_band_missing'] = df_prep['imd_band'].isna().astype(int)
    df_prep['imd_band'] = df_prep['imd_band'].fillna('Unknown')
    print("✓ imd_band: Created missing indicator, filled with 'Unknown'")

# 2b. Fill VLE columns with 0
vle_cols = [c for c in df_prep.columns if c.startswith('vle_')]
df_prep[vle_cols] = df_prep[vle_cols].fillna(0)
print(f"✓ VLE columns ({len(vle_cols)}): Filled with 0")

# 2c. Fill assessment columns with 0
assess_cols = [c for c in df_prep.columns if c.startswith('assess_')]
df_prep[assess_cols] = df_prep[assess_cols].fillna(0)
print(f"✓ Assessment columns ({len(assess_cols)}): Filled with 0")

# 2d. Impute remaining numerical with median using SimpleImputer
num_cols_remaining = df_prep.select_dtypes(include=[np.number]).columns
cols_with_na = [c for c in num_cols_remaining if df_prep[c].isna().any()]

if cols_with_na:
    imputer = SimpleImputer(strategy='median')
    df_prep[cols_with_na] = imputer.fit_transform(df_prep[cols_with_na])
    print(f"✓ Remaining numerical ({len(cols_with_na)}): Imputed with median")

print(f"\n✓ Remaining missing values: {df_prep.isna().sum().sum()}")
print("-" * 60)

## 4.3 Handle Outliers

In [ ]:
# =============================================================================
# HANDLE OUTLIERS (CAPPING)
# =============================================================================
# Strategy: Cap extreme values at 99th percentile
# This preserves information while reducing impact of extreme outliers
# =============================================================================

print("Step 3: Handle outliers (capping at 99th percentile)")
print("-" * 60)

# Columns to cap (engagement metrics with extreme outliers)
cols_to_cap = ['vle_total_clicks', 'vle_active_days', 'vle_early_clicks', 'vle_unique_resources']
cols_to_cap = [c for c in cols_to_cap if c in df_prep.columns]

for col in cols_to_cap:
    p99 = df_prep[col].quantile(0.99)
    n_capped = (df_prep[col] > p99).sum()
    df_prep[col] = df_prep[col].clip(upper=p99)
    print(f"✓ {col}: Capped {n_capped} values at {p99:.0f}")

print("-" * 60)

## 4.4 Feature Engineering - Derived Features

In [ ]:
# =============================================================================
# FEATURE ENGINEERING: RATIO FEATURES
# =============================================================================
# Create new features by combining existing ones
# Ratios often capture meaningful patterns
# =============================================================================

print("Step 4: Feature Engineering - Ratio Features")
print("-" * 60)

# Clicks per active day (engagement intensity)
df_prep['clicks_per_day'] = df_prep['vle_total_clicks'] / df_prep['vle_active_days'].replace(0, 1)
print("✓ Created: clicks_per_day (total_clicks / active_days)")

# Score per assessment (average performance)
if 'assess_count' in df_prep.columns and 'assess_score_mean' in df_prep.columns:
    df_prep['assessments_completed'] = df_prep['assess_count']
    print("✓ Created: assessments_completed")

# Early engagement ratio (proportion of total engagement that happened early)
if 'vle_early_clicks' in df_prep.columns:
    df_prep['early_engagement_ratio'] = df_prep['vle_early_clicks'] / df_prep['vle_total_clicks'].replace(0, 1)
    print("✓ Created: early_engagement_ratio (early_clicks / total_clicks)")

# Resources per active day
if 'vle_unique_resources' in df_prep.columns:
    df_prep['resources_per_day'] = df_prep['vle_unique_resources'] / df_prep['vle_active_days'].replace(0, 1)
    print("✓ Created: resources_per_day")

print("-" * 60)

In [ ]:
# =============================================================================
# FEATURE ENGINEERING: BINARY FLAGS
# =============================================================================
# Create yes/no indicators for specific conditions
# =============================================================================

print("Step 5: Feature Engineering - Binary Flags")
print("-" * 60)

# Is active early? (any engagement in first 2 weeks)
if 'vle_early_clicks' in df_prep.columns:
    df_prep['is_active_early'] = (df_prep['vle_early_clicks'] > 0).astype(int)
    print("✓ Created: is_active_early (1 if any early clicks, 0 otherwise)")

# Registered early? (before course start)
if 'date_registration' in df_prep.columns:
    df_prep['registered_early'] = (df_prep['date_registration'] < 0).astype(int)
    print("✓ Created: registered_early (1 if registered before day 0)")

# Has submitted any assessment?
if 'assess_count' in df_prep.columns:
    df_prep['has_submitted'] = (df_prep['assess_count'] > 0).astype(int)
    print("✓ Created: has_submitted (1 if any assessments submitted)")

# High performer? (average score > 70)
if 'assess_score_mean' in df_prep.columns:
    df_prep['is_high_performer'] = (df_prep['assess_score_mean'] >= 70).astype(int)
    print("✓ Created: is_high_performer (1 if avg score >= 70)")

# Has previous attempts?
if 'num_of_prev_attempts' in df_prep.columns:
    df_prep['has_prev_attempts'] = (df_prep['num_of_prev_attempts'] > 0).astype(int)
    print("✓ Created: has_prev_attempts")

print("-" * 60)

## 4.5 Feature Engineering - Binning with pd.cut()

In [ ]:
# =============================================================================
# FEATURE ENGINEERING: BINNING WITH pd.cut()
# =============================================================================
# Convert continuous variables into categorical bins
# This can capture non-linear relationships
# =============================================================================

print("Step 6: Feature Engineering - Binning with pd.cut()")
print("-" * 60)

# Engagement level bins (based on total clicks)
df_prep['engagement_level'] = pd.cut(
    df_prep['vle_total_clicks'],
    bins=[0, 100, 500, 1000, 2000, float('inf')],
    labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'],
    include_lowest=True
)
print("✓ Created: engagement_level (binned total_clicks)")
print(f"   Distribution: {df_prep['engagement_level'].value_counts().to_dict()}")

# Score level bins
df_prep['score_level'] = pd.cut(
    df_prep['assess_score_mean'],
    bins=[0, 40, 60, 70, 80, 100],
    labels=['Failing', 'Poor', 'Average', 'Good', 'Excellent'],
    include_lowest=True
)
print("\n✓ Created: score_level (binned assess_score_mean)")
print(f"   Distribution: {df_prep['score_level'].value_counts().to_dict()}")

# Active days bins
df_prep['activity_level'] = pd.cut(
    df_prep['vle_active_days'],
    bins=[0, 10, 30, 60, 100, float('inf')],
    labels=['Minimal', 'Low', 'Moderate', 'Regular', 'Intensive'],
    include_lowest=True
)
print("\n✓ Created: activity_level (binned active_days)")
print(f"   Distribution: {df_prep['activity_level'].value_counts().to_dict()}")

print("-" * 60)

In [ ]:
# =============================================================================
# VISUALIZE BINNED FEATURES VS OUTCOME
# =============================================================================

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

binned_features = ['engagement_level', 'score_level', 'activity_level']

for i, feat in enumerate(binned_features):
    ax = axes[i]
    ct = pd.crosstab(df_prep[feat], df_prep['final_result'], normalize='index') * 100
    order = [c for c in ['Pass', 'Distinction', 'Fail', 'Withdrawn'] if c in ct.columns]
    ct[order].plot(kind='bar', stacked=True, ax=ax,
                   color=['#2ecc71', '#3498db', '#e74c3c', '#95a5a6'])
    ax.set_title(f'Outcome by {feat}', fontweight='bold')
    ax.set_ylabel('Percentage')
    ax.tick_params(axis='x', rotation=45)
    ax.legend(loc='upper right', fontsize=8)

plt.tight_layout()
plt.show()

print("📝 Observation: Clear relationship between engagement/score levels and outcomes")

## 4.6 Feature Engineering - Interaction Features

In [ ]:
# =============================================================================
# FEATURE ENGINEERING: INTERACTION FEATURES
# =============================================================================
# Combine features to capture combined effects
# =============================================================================

print("Step 7: Feature Engineering - Interaction Features")
print("-" * 60)

# Engagement × Score (comprehensive performance)
df_prep['engagement_score_product'] = df_prep['vle_total_clicks'] * df_prep['assess_score_mean'] / 1000
print("✓ Created: engagement_score_product (clicks × score / 1000)")

# Early engagement × has submitted
if 'vle_early_clicks' in df_prep.columns and 'has_submitted' in df_prep.columns:
    df_prep['early_submission_indicator'] = df_prep['vle_early_clicks'] * df_prep['has_submitted']
    print("✓ Created: early_submission_indicator (early_clicks × has_submitted)")

# Active days × Score
df_prep['consistency_score'] = df_prep['vle_active_days'] * df_prep['assess_score_mean'] / 100
print("✓ Created: consistency_score (active_days × score / 100)")

print("-" * 60)

## 4.7 Encode Categorical Variables

In [ ]:
# =============================================================================
# ENCODE CATEGORICAL VARIABLES
# =============================================================================
# Strategy:
# 1. One-hot encoding for nominal categories (no order)
# 2. Ordinal encoding for ordered categories
# =============================================================================

print("Step 8: Encode categorical variables")
print("-" * 60)

# 8a. One-hot encoding for nominal variables (using pd.get_dummies with drop_first=True)
nominal_cols = ['gender', 'disability']
for col in nominal_cols:
    if col in df_prep.columns:
        dummies = pd.get_dummies(df_prep[col], prefix=col, drop_first=True)
        df_prep = pd.concat([df_prep, dummies], axis=1)
        print(f"✓ One-hot encoded: {col} → {list(dummies.columns)}")

# 8b. Ordinal encoding for education (has natural order)
education_order = {
    'No Formal quals': 0,
    'Lower Than A Level': 1,
    'A Level or Equivalent': 2,
    'HE Qualification': 3,
    'Post Graduate Qualification': 4
}
if 'highest_education' in df_prep.columns:
    df_prep['education_level'] = df_prep['highest_education'].map(education_order)
    print(f"✓ Ordinal encoded: highest_education → education_level")

# 8c. Ordinal encoding for age band
age_order = {'0-35': 0, '35-55': 1, '55<=': 2}
if 'age_band' in df_prep.columns:
    df_prep['age_level'] = df_prep['age_band'].map(age_order)
    print(f"✓ Ordinal encoded: age_band → age_level")

# 8d. Ordinal encoding for binned features
engagement_order = {'Very Low': 0, 'Low': 1, 'Medium': 2, 'High': 3, 'Very High': 4}
df_prep['engagement_level_encoded'] = df_prep['engagement_level'].map(engagement_order)
print(f"✓ Ordinal encoded: engagement_level → engagement_level_encoded")

score_order = {'Failing': 0, 'Poor': 1, 'Average': 2, 'Good': 3, 'Excellent': 4}
df_prep['score_level_encoded'] = df_prep['score_level'].map(score_order)
print(f"✓ Ordinal encoded: score_level → score_level_encoded")

activity_order = {'Minimal': 0, 'Low': 1, 'Moderate': 2, 'Regular': 3, 'Intensive': 4}
df_prep['activity_level_encoded'] = df_prep['activity_level'].map(activity_order)
print(f"✓ Ordinal encoded: activity_level → activity_level_encoded")

print("-" * 60)

## 4.8 Create Target Variable

In [ ]:
# =============================================================================
# CREATE BINARY TARGET VARIABLE
# =============================================================================
# 1 = Success (Pass or Distinction)
# 0 = Failure (Fail or Withdrawn)
# =============================================================================

print("Step 9: Create target variable")
print("-" * 60)

df_prep['target'] = df_prep['final_result'].apply(
    lambda x: 1 if x in ['Pass', 'Distinction'] else 0
)

print("Target distribution:")
print(df_prep['target'].value_counts())
print(f"\nSuccess rate: {df_prep['target'].mean()*100:.1f}%")
print("-" * 60)

## 4.9 Feature Selection

In [ ]:
# =============================================================================
# IDENTIFY ALL CANDIDATE FEATURES
# =============================================================================

print("Step 10: Identify candidate features")
print("-" * 60)

# Columns to EXCLUDE from features
exclude_cols = [
    'id_student', 'student_module_key', 'code_module', 'code_presentation',
    'gender', 'region', 'highest_education', 'imd_band', 'age_band', 'disability',
    'final_result', 'target', 'target_binary',
    'date_registration', 'date_unregistration',
    'engagement_level', 'score_level', 'activity_level'  # Keep only encoded versions
]

# Get all numerical columns as candidates
candidate_features = [col for col in df_prep.columns 
                      if col not in exclude_cols 
                      and df_prep[col].dtype in ['int64', 'float64', 'int32', 'float32', 'uint8', 'bool']]

print(f"Total candidate features: {len(candidate_features)}")
print("-" * 60)

In [ ]:
# =============================================================================
# SELECT TOP K FEATURES BY CORRELATION
# =============================================================================

print("Step 11: Select top K features by correlation with target")
print("-" * 60)

# Calculate correlations
correlations = df_prep[candidate_features + ['target']].corr()['target'].drop('target')
correlations_abs = correlations.abs().sort_values(ascending=False)

print("Top 20 features by |correlation| with target:")
for i, (feat, corr) in enumerate(correlations_abs.head(20).items(), 1):
    direction = '+' if correlations[feat] > 0 else '-'
    print(f"  {i:2}. {feat}: {direction}{corr:.4f}")
print("-" * 60)

In [ ]:
# =============================================================================
# SELECT FINAL FEATURES
# =============================================================================

K = 15  # Number of features to use (adjustable)

selected_features = correlations_abs.head(K).index.tolist()

print(f"\n✓ SELECTED TOP {K} FEATURES FOR MODELLING:")
print("=" * 60)
for i, feat in enumerate(selected_features, 1):
    corr = correlations[feat]
    print(f"  {i:2}. {feat} (r = {corr:+.4f})")
print("=" * 60)

In [ ]:
# =============================================================================
# VISUALIZE SELECTED FEATURES CORRELATION
# =============================================================================

plt.figure(figsize=(12, 10))
corr_matrix = df_prep[selected_features + ['target']].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f', square=True)
plt.title(f'Correlation Matrix (Selected {K} Features + Target)', fontweight='bold')
plt.tight_layout()
plt.show()

## 4.10 Prepare Final Dataset

In [ ]:
# =============================================================================
# CREATE FINAL FEATURE MATRIX AND TARGET
# =============================================================================

print("Step 12: Create final dataset")
print("-" * 60)

X = df_prep[selected_features].copy()
y = df_prep['target'].copy()

# Handle any remaining infinities
X = X.replace([np.inf, -np.inf], np.nan).fillna(0)

print(f"Feature matrix X: {X.shape[0]:,} rows × {X.shape[1]} features")
print(f"Target vector y: {y.shape[0]:,} values")
print(f"\nFeatures: {selected_features}")
print("-" * 60)

## 4.11 Data Preparation Summary

In [ ]:
# =============================================================================
# DATA PREPARATION SUMMARY
# =============================================================================

print("=" * 70)
print("                    DATA PREPARATION SUMMARY")
print("=" * 70)

print(f"""
📊 ORIGINAL DATA
─────────────────
• Rows: {df.shape[0]:,}
• Columns: {df.shape[1]}

🔧 TRANSFORMATIONS APPLIED
───────────────────────────
1. Missing Values:
   - Created imd_band_missing indicator
   - Filled VLE/assessment with 0
   - Imputed numerical with median

2. Outlier Handling:
   - Capped at 99th percentile

3. Feature Engineering:
   - Ratio features: clicks_per_day, early_engagement_ratio, resources_per_day
   - Binary flags: is_active_early, registered_early, has_submitted, is_high_performer
   - Binning: engagement_level, score_level, activity_level
   - Interactions: engagement_score_product, consistency_score

4. Encoding:
   - One-hot: gender, disability
   - Ordinal: education_level, age_level, binned features

5. Feature Selection:
   - Selected top {K} features by correlation

📈 FINAL DATASET
─────────────────
• Rows: {X.shape[0]:,}
• Features: {X.shape[1]}
• Target: Binary (1=Success, 0=Failure)
""")
print("=" * 70)

---

# 5️⃣ Phase 4: Modelling - Task 1 (Classification)

## 5.1 Train-Test Split

In [ ]:
# =============================================================================
# TRAIN-TEST SPLIT
# =============================================================================
# Stratified split to maintain class proportions in both sets
# =============================================================================

print("Step 13: Train-test split (stratified)")
print("-" * 60)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.0f}%)")
print(f"Test set: {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.0f}%)")
print(f"Features: {X_train.shape[1]}")
print(f"\nTarget distribution (train):")
print(f"  Success (1): {y_train.sum():,} ({y_train.mean()*100:.1f}%)")
print(f"  Failure (0): {(y_train==0).sum():,} ({(1-y_train.mean())*100:.1f}%)")
print("-" * 60)

## 5.2 Feature Scaling

In [ ]:
# =============================================================================
# FEATURE SCALING WITH StandardScaler
# =============================================================================
# Transforms features to have mean=0 and std=1
# Important for: Logistic Regression, SVM, Neural Networks
# Not needed for: Tree-based models (RF, GB)
# =============================================================================

print("Step 14: Feature scaling (StandardScaler)")
print("-" * 60)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✓ Features scaled (mean=0, std=1)")
print("  Used for: Logistic Regression")
print("  Not needed for: Random Forest, Gradient Boosting")
print("-" * 60)

## 5.3 Train Multiple Models

In [ ]:
# =============================================================================
# TRAIN AND EVALUATE MULTIPLE MODELS
# =============================================================================

print("Step 15: Train and evaluate models")
print("-" * 60)

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42)
}

results = {}

for name, model in models.items():
    print(f"\n🔄 Training {name}...")
    
    # Use scaled data for Logistic, unscaled for tree models
    if 'Logistic' in name:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_proba = model.predict_proba(X_test_scaled)[:, 1]
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]
    
    # Calculate metrics
    results[name] = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1': f1_score(y_test, y_pred),
        'AUC-ROC': roc_auc_score(y_test, y_proba)
    }
    
    print(f"   ✓ Accuracy: {results[name]['Accuracy']:.4f}")
    print(f"   ✓ AUC-ROC:  {results[name]['AUC-ROC']:.4f}")

print("\n" + "-" * 60)

## 5.4 Model Comparison

In [ ]:
# =============================================================================
# COMPARE MODEL PERFORMANCE
# =============================================================================

print("Step 16: Model comparison")
print("-" * 60)

results_df = pd.DataFrame(results).T
print(results_df.round(4))

best_model_name = results_df['AUC-ROC'].idxmax()
print(f"\n🏆 Best model (by AUC-ROC): {best_model_name}")
print("-" * 60)

In [ ]:
# Visualize comparison
fig, ax = plt.subplots(figsize=(12, 5))
results_df.plot(kind='bar', ax=ax)
ax.set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
ax.set_ylabel('Score')
ax.set_xticklabels(results_df.index, rotation=0)
ax.legend(loc='lower right')
ax.set_ylim(0, 1)
plt.tight_layout()
plt.show()

## 5.5 Best Model Detailed Analysis

In [ ]:
# =============================================================================
# DETAILED ANALYSIS OF BEST MODEL
# =============================================================================

print("Step 17: Detailed analysis - Random Forest")
print("-" * 60)

rf = models['Random Forest']
y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:, 1]

print("Classification Report:")
print(classification_report(y_test, y_pred_rf, 
                            target_names=['Fail/Withdrawn', 'Pass/Distinction']))
print("-" * 60)

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_rf)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Fail/Withdrawn', 'Pass/Distinction'],
            yticklabels=['Fail/Withdrawn', 'Pass/Distinction'])
plt.title('Confusion Matrix - Random Forest', fontsize=14, fontweight='bold')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.tight_layout()
plt.show()

tn, fp, fn, tp = cm.ravel()
print(f"True Negatives (correct failures): {tn:,}")
print(f"True Positives (correct successes): {tp:,}")
print(f"False Positives (predicted success, was failure): {fp:,}")
print(f"False Negatives (predicted failure, was success): {fn:,}")

## 5.6 Feature Importance

In [ ]:
# =============================================================================
# FEATURE IMPORTANCE ANALYSIS
# =============================================================================

print("Step 18: Feature importance")
print("-" * 60)

importance = pd.DataFrame({
    'Feature': selected_features,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)

print("Feature importance ranking:")
for i, row in importance.iterrows():
    print(f"  {row['Feature']}: {row['Importance']:.4f}")
print("-" * 60)

In [ ]:
# Visualize
plt.figure(figsize=(10, 8))
plt.barh(range(len(importance)), importance['Importance'], color='steelblue')
plt.yticks(range(len(importance)), importance['Feature'])
plt.xlabel('Importance')
plt.title('Feature Importance (Random Forest)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 5.7 ROC Curves

In [ ]:
# =============================================================================
# ROC CURVES COMPARISON
# =============================================================================

print("Step 19: ROC curves")
print("-" * 60)

plt.figure(figsize=(10, 8))

colors = ['blue', 'green', 'red']
for (name, model), color in zip(models.items(), colors):
    if 'Logistic' in name:
        y_proba = model.predict_proba(X_test_scaled)[:, 1]
    else:
        y_proba = model.predict_proba(X_test)[:, 1]
    
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    auc = roc_auc_score(y_test, y_proba)
    plt.plot(fpr, tpr, label=f'{name} (AUC={auc:.3f})', color=color, linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', label='Random (AUC=0.500)')
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves Comparison', fontsize=14, fontweight='bold')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5.8 Hyperparameter Tuning

In [ ]:
# =============================================================================
# HYPERPARAMETER TUNING WITH GRIDSEARCHCV
# =============================================================================

print("Step 20: Hyperparameter tuning (GridSearchCV)")
print("-" * 60)

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5]
}

print("Parameter grid:")
for param, values in param_grid.items():
    print(f"  {param}: {values}")

print("\n🔄 Running GridSearchCV (this may take a few minutes)...")

grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1),
    param_grid,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)
grid_search.fit(X_train, y_train)

print(f"\n✓ Best parameters: {grid_search.best_params_}")
print(f"✓ Best CV AUC-ROC: {grid_search.best_score_:.4f}")
print("-" * 60)

In [ ]:
# Evaluate tuned model
print("Tuned model performance on test set:")
print("-" * 60)

best_rf = grid_search.best_estimator_
y_pred_tuned = best_rf.predict(X_test)
y_proba_tuned = best_rf.predict_proba(X_test)[:, 1]

print(f"Accuracy:  {accuracy_score(y_test, y_pred_tuned):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_tuned):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred_tuned):.4f}")
print(f"F1 Score:  {f1_score(y_test, y_pred_tuned):.4f}")
print(f"AUC-ROC:   {roc_auc_score(y_test, y_proba_tuned):.4f}")
print("-" * 60)

## 5.9 Cross-Validation

In [ ]:
# =============================================================================
# CROSS-VALIDATION FOR ROBUST EVALUATION
# =============================================================================

print("Step 21: 5-Fold Cross-Validation")
print("-" * 60)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(best_rf, X, y, cv=cv, scoring='roc_auc')

print(f"CV AUC-ROC scores: {cv_scores.round(4)}")
print(f"Mean: {cv_scores.mean():.4f}")
print(f"Std:  {cv_scores.std():.4f}")
print(f"95% CI: [{cv_scores.mean() - 1.96*cv_scores.std():.4f}, {cv_scores.mean() + 1.96*cv_scores.std():.4f}]")
print("-" * 60)

---

# 6️⃣ Phase 4: Modelling - Task 2 (Clustering)

## 6.1 Select Clustering Features

In [ ]:
# =============================================================================
# SELECT FEATURES FOR CLUSTERING
# =============================================================================
# Use engagement-focused features to segment students by behavior
# =============================================================================

print("Step 22: Select clustering features")
print("-" * 60)

cluster_features = [
    'vle_total_clicks', 'vle_active_days', 'vle_unique_resources',
    'assess_score_mean', 'assess_count', 'clicks_per_day'
]
cluster_features = [f for f in cluster_features if f in df_prep.columns]

print(f"Using {len(cluster_features)} features for clustering:")
for f in cluster_features:
    print(f"  - {f}")
print("-" * 60)

In [ ]:
# Prepare and scale
X_cluster = df_prep[cluster_features].fillna(0)
X_cluster = X_cluster.replace([np.inf, -np.inf], 0)

scaler_cluster = StandardScaler()
X_cluster_scaled = scaler_cluster.fit_transform(X_cluster)

print(f"Clustering data: {X_cluster_scaled.shape}")

## 6.2 Find Optimal Number of Clusters

In [ ]:
# =============================================================================
# EVALUATE DIFFERENT K VALUES
# =============================================================================

print("Step 23: Evaluate k values (2-10)")
print("-" * 60)

K_range = range(2, 11)
inertias = []
silhouettes = []
db_scores = []

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_cluster_scaled)
    inertias.append(kmeans.inertia_)
    silhouettes.append(silhouette_score(X_cluster_scaled, labels))
    db_scores.append(davies_bouldin_score(X_cluster_scaled, labels))
    print(f"k={k}: Silhouette={silhouettes[-1]:.4f}, Davies-Bouldin={db_scores[-1]:.4f}")

print("-" * 60)

In [ ]:
# Visualize
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(K_range, inertias, 'bo-', linewidth=2)
axes[0].set_xlabel('Number of Clusters (k)')
axes[0].set_ylabel('Inertia')
axes[0].set_title('Elbow Method', fontweight='bold')
axes[0].grid(True, alpha=0.3)

axes[1].plot(K_range, silhouettes, 'go-', linewidth=2)
axes[1].set_xlabel('Number of Clusters (k)')
axes[1].set_ylabel('Silhouette Score')
axes[1].set_title('Silhouette (higher = better)', fontweight='bold')
axes[1].grid(True, alpha=0.3)

axes[2].plot(K_range, db_scores, 'ro-', linewidth=2)
axes[2].set_xlabel('Number of Clusters (k)')
axes[2].set_ylabel('Davies-Bouldin Index')
axes[2].set_title('Davies-Bouldin (lower = better)', fontweight='bold')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6.3 Fit Final K-Means Model

In [ ]:
# =============================================================================
# FIT FINAL K-MEANS MODEL
# =============================================================================

print("Step 24: Fit K-Means with optimal k")
print("-" * 60)

OPTIMAL_K = 4  # Based on elbow/silhouette analysis

kmeans_final = KMeans(n_clusters=OPTIMAL_K, random_state=42, n_init=10)
cluster_labels = kmeans_final.fit_predict(X_cluster_scaled)

df_prep['cluster'] = cluster_labels

print(f"K-Means with k={OPTIMAL_K}:")
cluster_counts = pd.Series(cluster_labels).value_counts().sort_index()
for c, count in cluster_counts.items():
    pct = count / len(cluster_labels) * 100
    print(f"  Cluster {c}: {count:,} students ({pct:.1f}%)")

print(f"\nSilhouette Score: {silhouette_score(X_cluster_scaled, cluster_labels):.4f}")
print(f"Davies-Bouldin Index: {davies_bouldin_score(X_cluster_scaled, cluster_labels):.4f}")
print("-" * 60)

## 6.4 Cluster Profiling

In [ ]:
# =============================================================================
# PROFILE EACH CLUSTER
# =============================================================================

print("Step 25: Cluster profiling")
print("-" * 60)

profiles = df_prep.groupby('cluster')[cluster_features].mean()
print("Cluster centroids (mean values):")
print(profiles.round(2).T)
print("-" * 60)

In [ ]:
# Visualize profiles
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, feat in enumerate(cluster_features[:6]):
    ax = axes[i]
    df_prep.boxplot(column=feat, by='cluster', ax=ax)
    ax.set_title(f'{feat} by Cluster', fontweight='bold')
    ax.set_xlabel('Cluster')
    plt.suptitle('')

plt.tight_layout()
plt.show()

## 6.5 Cluster vs Outcome Analysis

In [ ]:
# =============================================================================
# CLUSTER VS OUTCOME ANALYSIS
# =============================================================================

print("Step 26: Cluster vs outcome")
print("-" * 60)

# Crosstab
crosstab = pd.crosstab(df_prep['cluster'], df_prep['final_result'], normalize='index') * 100
print("Outcome distribution by cluster (%):")
print(crosstab.round(1))

# Success rate
success_rate = df_prep.groupby('cluster')['target'].mean() * 100
print("\nSuccess rate by cluster:")
for c, rate in success_rate.items():
    risk = 'HIGH RISK' if rate < 50 else 'MEDIUM RISK' if rate < 70 else 'LOW RISK'
    print(f"  Cluster {c}: {rate:.1f}% - {risk}")
print("-" * 60)

In [ ]:
# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Stacked outcomes
order = ['Pass', 'Distinction', 'Fail', 'Withdrawn']
cols = [c for c in order if c in crosstab.columns]
crosstab[cols].plot(kind='bar', stacked=True, ax=axes[0],
                    color=['#2ecc71', '#3498db', '#e74c3c', '#95a5a6'])
axes[0].set_title('Outcomes by Cluster', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Percentage')
axes[0].tick_params(axis='x', rotation=0)
axes[0].legend(title='Final Result')

# Success rate
colors = ['#e74c3c' if r < 50 else '#f39c12' if r < 70 else '#2ecc71' for r in success_rate]
success_rate.plot(kind='bar', ax=axes[1], color=colors, edgecolor='black')
axes[1].set_title('Success Rate by Cluster', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Success Rate (%)')
axes[1].tick_params(axis='x', rotation=0)
axes[1].axhline(df_prep['target'].mean()*100, color='black', linestyle='--', label='Overall')
axes[1].legend()

for i, v in enumerate(success_rate):
    axes[1].text(i, v + 1, f'{v:.0f}%', ha='center', fontsize=11)

plt.tight_layout()
plt.show()

## 6.6 PCA Visualization

In [ ]:
# =============================================================================
# PCA VISUALIZATION
# =============================================================================

print("Step 27: PCA visualization")
print("-" * 60)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_cluster_scaled)

print(f"Variance explained: PC1={pca.explained_variance_ratio_[0]:.1%}, PC2={pca.explained_variance_ratio_[1]:.1%}")
print(f"Total: {sum(pca.explained_variance_ratio_):.1%}")

plt.figure(figsize=(12, 10))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=cluster_labels, cmap='viridis', alpha=0.5, s=10)
plt.colorbar(scatter, label='Cluster')

# Centroids
centers_pca = pca.transform(kmeans_final.cluster_centers_)
plt.scatter(centers_pca[:, 0], centers_pca[:, 1], c='red', marker='X', s=300, 
            edgecolors='black', linewidths=2, label='Centroids')

for i, (x, y_coord) in enumerate(centers_pca):
    plt.annotate(f'C{i}', (x, y_coord), fontsize=12, fontweight='bold', ha='center', va='bottom', color='red')

plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)', fontsize=12)
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)', fontsize=12)
plt.title('Student Clusters (PCA Visualization)', fontsize=14, fontweight='bold')
plt.legend()
plt.tight_layout()
plt.show()

## 6.7 Cluster Interpretation

In [ ]:
# =============================================================================
# INTERPRET AND NAME CLUSTERS
# =============================================================================

print("Step 28: Cluster interpretation")
print("=" * 60)

for c in range(OPTIMAL_K):
    profile = profiles.loc[c]
    rate = success_rate[c]
    count = cluster_counts[c]
    
    print(f"\n🔹 CLUSTER {c}")
    print(f"   Size: {count:,} ({count/len(df_prep)*100:.1f}%)")
    print(f"   Success Rate: {rate:.1f}%")
    print(f"   Avg Clicks: {profile.get('vle_total_clicks', 0):.0f}")
    print(f"   Avg Active Days: {profile.get('vle_active_days', 0):.0f}")
    print(f"   Avg Score: {profile.get('assess_score_mean', 0):.1f}")

print("\n" + "=" * 60)
print("\n📝 Suggested cluster names (based on analysis):")
print("   Example names: 'Disengaged', 'Late Starters', 'High Achievers', 'Consistent Learners'")
print("   ➜ Update based on YOUR actual cluster profiles above!")

---

# 7️⃣ Phase 5: Evaluation Summary

In [ ]:
print("=" * 70)
print("                         EVALUATION SUMMARY")
print("=" * 70)

print("\n" + "-" * 70)
print("TASK 1: CLASSIFICATION (Predictive Model)")
print("-" * 70)
print(f"Model: Random Forest (Tuned)")
print(f"Features: {K} (selected by correlation)")
print(f"\nTest Set Performance:")
print(f"  • Accuracy:  {accuracy_score(y_test, y_pred_tuned):.4f}")
print(f"  • Precision: {precision_score(y_test, y_pred_tuned):.4f}")
print(f"  • Recall:    {recall_score(y_test, y_pred_tuned):.4f}")
print(f"  • F1 Score:  {f1_score(y_test, y_pred_tuned):.4f}")
print(f"  • AUC-ROC:   {roc_auc_score(y_test, y_proba_tuned):.4f}")

print(f"\nCross-Validation (5-fold):")
print(f"  • Mean AUC-ROC: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

print(f"\nTop 5 Predictive Features:")
for _, row in importance.head(5).iterrows():
    print(f"  • {row['Feature']}: {row['Importance']:.4f}")

print("\n" + "-" * 70)
print("TASK 2: CLUSTERING (Segmentation Model)")
print("-" * 70)
print(f"Algorithm: K-Means")
print(f"Number of Clusters: {OPTIMAL_K}")
print(f"Features: {len(cluster_features)}")
print(f"Silhouette Score: {silhouette_score(X_cluster_scaled, cluster_labels):.4f}")

print(f"\nCluster Risk Levels:")
for c in range(OPTIMAL_K):
    rate = success_rate[c]
    risk = 'HIGH RISK' if rate < 50 else 'MEDIUM' if rate < 70 else 'LOW RISK'
    print(f"  Cluster {c}: {cluster_counts[c]:,} students, {rate:.1f}% success → {risk}")

print("\n" + "-" * 70)
print("SUCCESS CRITERIA CHECK")
print("-" * 70)
auc_val = roc_auc_score(y_test, y_proba_tuned)
acc_val = accuracy_score(y_test, y_pred_tuned)
sil_val = silhouette_score(X_cluster_scaled, cluster_labels)

print(f"  {'✓' if auc_val > 0.75 else '✗'} AUC-ROC > 0.75: {auc_val:.4f}")
print(f"  {'✓' if acc_val > 0.70 else '✗'} Accuracy > 70%: {acc_val*100:.1f}%")
print(f"  {'✓' if sil_val > 0.2 else '✗'} Silhouette > 0.2: {sil_val:.4f}")
print("=" * 70)

---

# 8️⃣ Phase 6: Deployment Recommendations

In [ ]:
print("=" * 70)
print("                    DEPLOYMENT RECOMMENDATIONS")
print("=" * 70)

print("""
📋 EARLY WARNING SYSTEM
─────────────────────────
1. Deploy the Random Forest model to predict at-risk students
2. Run predictions WEEKLY during the first 4 weeks of term
3. Flag students with P(success) < 0.5 for intervention
4. Prioritize based on early engagement metrics

📊 KEY PREDICTIVE INDICATORS
─────────────────────────────
Based on feature importance, focus on monitoring:
""")

for _, row in importance.head(5).iterrows():
    print(f"  • {row['Feature']}")

print("""
👥 CLUSTER-BASED INTERVENTIONS
───────────────────────────────""")

for c in range(OPTIMAL_K):
    rate = success_rate[c]
    if rate < 50:
        print(f"  Cluster {c} (HIGH RISK - {rate:.0f}%):")
        print(f"    → Immediate personal tutor contact")
        print(f"    → One-on-one study support")
    elif rate < 70:
        print(f"  Cluster {c} (MEDIUM RISK - {rate:.0f}%):")
        print(f"    → Group study skills workshops")
        print(f"    → Peer mentoring programs")
    else:
        print(f"  Cluster {c} (LOW RISK - {rate:.0f}%):")
        print(f"    → Light-touch monitoring")
        print(f"    → Enrichment opportunities")

print("""
⚠️ LIMITATIONS
──────────────
• Model trained on historical data - may not generalize to new courses
• Requires real-time VLE data access for predictions
• Cannot capture external factors (health, family, work)
• Should be validated each semester with new data

🔄 MONITORING PLAN
───────────────────
• Track model accuracy monthly
• Retrain model each academic year
• Collect feedback from tutors on intervention effectiveness
• A/B test interventions to measure impact

❓ QUESTIONS FOR STAKEHOLDER (AMIR)
────────────────────────────────────
1. What is the acceptable false positive rate for flagging students?
2. What resources are available for interventions?
3. Are there privacy concerns with predictive monitoring?
4. Should the model weight recall over precision?
""")
print("=" * 70)

---

# 📝 Summary

## CRISP-DM Phases Completed

| Phase | Status | Key Activities |
|-------|--------|----------------|
| 1. Business Understanding | ✓ | Defined objectives, success criteria |
| 2. Data Understanding | ✓ | EDA, outliers, distributions, correlations, scatter plots |
| 3. Data Preparation | ✓ | Missing values, outlier capping, feature engineering, encoding |
| 4. Modelling | ✓ | 3 classifiers, K-Means clustering, hyperparameter tuning |
| 5. Evaluation | ✓ | Metrics, CV, confusion matrix, cluster analysis |
| 6. Deployment | ✓ | Recommendations, limitations, monitoring plan |

## Key Decisions

1. **Feature Selection**: Top 15 features by correlation (not 89!)
2. **Feature Engineering**: Created ratio, binary, binned, and interaction features
3. **Model Choice**: Random Forest (best AUC-ROC)
4. **Clustering**: K=4 (based on elbow/silhouette analysis)

## Files Generated

- This notebook: `FAIDM_CRISP_DM_Analysis.ipynb`
- Prerequisite: `oulad_mega_table.csv` (from `Create_Mega_Table.ipynb`)